## import dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load Cats vs Dogs dataset with train, validation, test splits

In [2]:
(train, validation, test), metadata = tfds.load(
                                        'cats_vs_dogs',
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        with_info=True,
                                        as_supervised=True
                                      )
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(num_examples)
print(num_classes)

23262
2


# Resize and normalize images

In [3]:
IMAGE_SIZE = (224, 224)

def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return image, label

# Prepare batches for training, validation, and testing

In [4]:
BATCH_SIZE = 32

## Training Batch
train_batches = train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)

## Validation Batch
validation_batches = validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

## Test batch
test_batches = test.map(format_image).batch(1)

# Load MobileNetV2 base model without top layers, pretrained on ImageNet

In [5]:
base_model = tf.keras.applications.MobileNetV2(
        input_shape=IMAGE_SIZE + (3,),
        include_top=False,
        weights='imagenet',
        pooling='avg'  # Global average pooling
    )

    # Freeze the base model
base_model.trainable = False

    # Build the model
model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

# Compile and train

In [6]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SIZE + (3,),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Build the model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), # Add a GlobalAveragePooling2D layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=5, validation_data=validation_batches)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 58s 71ms/step - accuracy: 0.9528 - loss: 0.1153 - val_accuracy: 0.9819 - val_loss: 0.0479
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.9838 - loss: 0.0422 - val_accuracy: 0.9802 - val_loss: 0.0512
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 49ms/step - accuracy: 0.9877 - loss: 0.0364 - val_accuracy: 0.9819 - val_loss: 0.0511
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.9885 - loss: 0.0353 - val_accuracy: 0.9837 - val_loss: 0.0484
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 53ms/step - accuracy: 0.9885 - loss: 0.0325 - val_accuracy: 0.9802 - val_loss: 0.0574


# Save model in SavedModel format

In [7]:
PetVision_model = "/content/drive/MyDrive/PetVision-TFLite/pet_vision_model"
tf.saved_model.save(model, PetVision_model)

# Convert to TensorFlow Lite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_file = 'pet_vision_model.tflite'
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

# Load TFLite interpreter and allocate tensors

In [ ]:
interpreter = tf.lite.Interpreter(model_path = tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

# Run inference on first 100 test images using TFLite interpreter

In [ ]:
test_labels, test_imgs = [], []
for img, label in test_batches.take(100):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

# Calculate number of correct predictions out of 100

In [ ]:
score = 0
for item in range(0,99):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1
print("Out of 100 predictions I got " + str(score) + " correct")

# Define function to plot a single image with prediction info

In [ ]:
def plot_image(i, predictions, true_labels, images):
    predictions_array = np.array(predictions[i][0])
    true_label = true_labels[i]
    img = images[i][0]

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel(
        "Predicted: {} ({:2.0f}%)\nTrue: {}".format(
            predicted_label,
            100 * np.max(predictions_array),
            true_label
        ),
        color=color
    )


# Plot test images in a grid, 3 images per row, using existing plot_image function

In [ ]:
num_images = 60
images_per_row = 3
rows = num_images // images_per_row + int(num_images % images_per_row > 0)

plt.figure(figsize=(5 * images_per_row, 4 * rows))

for i in range(num_images):
    plt.subplot(rows, images_per_row, i + 1)
    plot_image(i, predictions, test_labels, test_imgs)

plt.tight_layout()
plt.show()